In [20]:
import simpy
import random

class icusim():
    def __init__(self) -> None:
        self.env = simpy.Environment()
        self.total_pacientes = 0
        self.pacientes_solicitacao = 0
        self.pacientes_internados = 0
        self.pacientes_atendidos = 0
        self.pacientes_perdidos = 0
        self.lista_pacientes = {}

        self.leitos = simpy.Resource(self.env, capacity=10)

    def paciente(self, nome, tempo_internacao):
        
        # tempo de chegada do paciente
        t0 = self.env.now
        self.adiciona_paciente(nome, t0=t0)
        
        with self.leitos.request() as req:
            yield req
            # tempo de possibilidade de vaga
            t1 = self.env.now
            if t1 - t0 < 36:
                # critério para internação
                self.pacientes_solicitacao -= 1
                self.adiciona_paciente(nome, t0=t0, t1=t1)
                self.pacientes_internados += 1
                yield self.env.timeout(tempo_internacao)
                
                # tempo de saida do paciente
                t2 = self.env.now
                self.pacientes_internados -= 1
                self.pacientes_atendidos += 1
            else:
                # critério para perda internação na uti
                self.pacientes_solicitacao -= 1
                self.pacientes_perdidos += 1
                t2 = -1
                


            self.adiciona_paciente(nome, t0=t0, t1=t1, t2=t2)
    
    def cria_paciente(self):
        paciente_nome = 0
        
        
        while True:
            self.total_pacientes += 1
            self.pacientes_solicitacao += 1
            # sorteia tempo para chegada de um novo paciente entre 2 a 12 hs
            yield self.env.timeout(random.randint(2, 12))
            
            tempo_internacao = random.randint(2*24, 10*24)       
            self.env.process(self.paciente(paciente_nome, tempo_internacao))
            
            paciente_nome += 1

    def adiciona_paciente(self, nome, t0, t1=None, t2=None):
        self.lista_pacientes[nome] = {'t0': t0, 't1': t1, 't2': t2}

hospital = icusim()
hospital.env.process(hospital.cria_paciente())
hospital.env.run(until=30*24)

In [21]:
print(hospital.total_pacientes, hospital.pacientes_perdidos+ hospital.pacientes_solicitacao+ hospital.pacientes_internados+ hospital.pacientes_atendidos)
print(f"Total de pacientes: {hospital.total_pacientes}")
print(f"Pacientes solicitações pendentes: {hospital.pacientes_solicitacao}")
print(f"Pacientes internados: {hospital.pacientes_internados}")
print(f"Pacientes atendidos: {hospital.pacientes_atendidos}")
print(f"Pacientes perdidos: {hospital.pacientes_perdidos}")


105 105
Total de pacientes: 105
Pacientes solicitações pendentes: 4
Pacientes internados: 10
Pacientes atendidos: 48
Pacientes perdidos: 43


In [13]:
for a in hospital.lista_pacientes:
    print(a, hospital.lista_pacientes[a]['t0'], hospital.lista_pacientes[a]['t1'], hospital.lista_pacientes[a]['t2'])

0 4 4 71
1 12 12 234
2 17 17 230
3 19 19 245
4 24 24 263
5 34 34 235
6 45 45 181
7 57 57 215
8 67 67 176
9 75 75 181
10 80 80 262
11 84 176 -1
12 86 176 -1
13 93 176 -1
14 98 176 -1
15 102 176 -1
16 108 176 -1
17 116 176 -1
18 123 176 -1
19 135 176 310
20 144 181 360
21 152 181 266
22 156 215 -1
23 158 215 -1
24 164 215 -1
25 172 215 402
26 183 230 305
27 189 234 462
28 201 235 440
29 203 245 403
30 210 262 -1
31 222 262 436
32 233 263 423
33 236 266 450
34 248 305 -1
35 259 305 438
36 264 310 445
37 269 360 -1
38 281 360 -1
39 288 360 -1
40 297 360 -1
41 300 360 -1
42 312 360 -1
43 317 360 427
44 322 402 -1
45 326 402 -1
46 336 402 -1
47 347 402 -1
48 352 402 -1
49 361 402 540
50 369 403 617
51 375 423 -1
52 384 423 563
53 389 427 513
54 395 436 518
55 401 438 532
56 412 440 542
57 420 445 506
58 432 450 573
59 439 462 597
60 446 506 -1
61 449 506 -1
62 459 506 642
63 471 513 622
64 483 518 658
65 488 532 None
66 500 540 607
67 511 542 None
68 518 563 677
69 522 573 -1
70 525 573 -1
7

411